In [ ]:
# python 3.6.8
# DLISIO v0.3.5
# numpy v1.16.2
# pandas v0.24.1
# lasio v0.25.1

from dlisio import lis
import pandas as pd
import os
import lasio
import numpy as np

In [ ]:
# If you don't care about speed, and need the well names extracted from the lis header, set the find_wellname = Yes
# Otherwise pass in the name of the well into the wellname variable.

def extract_wellname(f, find_wellname, manualwellname):
    if find_wellname == "Yes":
        records = f.wellsite_data()
        inforec = records[0] # assume well name in is in the first record
        #Can it be structured? Otherwise return an error
        if inforec.isstructured() == True:
            np_array = (inforec.table(simple=True)).T
            df = pd.DataFrame(data=np_array)
            
            wellname_list = []

            items = ['Well Name', 'wellname', 'WN', 'WELL NAME'] ##set all the possible search terms

            for item in items:
                df["string_index"] = df["VALU"].str.find(str(item)).values
                wellname = df.loc[df['string_index'] >= 0, 'VALU'].values.tolist()
                #wellname = wellname[0] #take the first value returned.
                wellname_list.append(wellname)

                df["string_index"] = df["MNEM"].str.find(str(item)).values
                wellname = df.loc[df['string_index'] >= 0, 'VALU'].values.tolist()
                wellname_list.append(wellname)

            wellname_list = [x for x in wellname_list if x != []]
            for wellname in wellname_list:
                wellname = [s.replace('Well Name:', '') for s in wellname]
                wellname = [s.replace(' WN', '') for s in wellname]
                wellname = [s.replace('Well Name ', '') for s in wellname]
                wellname = [s.replace('WN', '') for s in wellname]
                wellname = [s.strip() for s in wellname]
                
            wellname = wellname[0]   ##take the first item in the list
            
            if len(wellname)>0:
                wellname_report_error = ""
            else:
                wellname_report_error = "Could not read header - no name extraction"
        else:
            wellname_report_error = "Could not read header - not structured"
    else:
        wellname = manualwellname
    return wellname, wellname_report_error

In [ ]:
# process each logical file and each sampling rate (frame) at a time. 
# The goal with this function is to only read from the lis file once!
# The number of logical files times the number of groups of sample rates will determine the number of las files created.
# Curves with no matching frame sample rate will not be included!!  See Fast Channels in the LIS User Guides @dlisio.readthedocs

def read_curves_and_curve_meta(filepath, manualwellname = "set input", find_wellname = "Yes"):
    curve_units = []
    curve_names = []
    curve_frames = []
    frame_count = 0
    with lis.load(filepath) as files:
        for f in files:
            wellname = extract_wellname(f, find_wellname, manualwellname)
            for fs in f.data_format_specs():
                for frame in fs.sample_rates():
                    #create dataframe
                    units = []
                    curvenames = []
                    meta = lis.curves_metadata(fs, sample_rate=frame)
                    curves = lis.curves(f, fs, sample_rate=frame)
                    curves_df = curves.T
                    names = curves.dtype.names
                    curves_df = pd.DataFrame(data=curves_df, columns=names)
                    
                    #get inventories
                    for n in names:
                        spec = meta[n]
                        units.append(spec.units)
                        curvenames.append(n)
                    curve_names.append(curvenames)
                    curve_units.append(units)
                    
                    # advance the count of sample rate frames
                    frame_count = frame_count + 1
                
                    #Quick quality test
                    unit_len = len(curve_units)
                    name_len = len(curve_names)
                    if unit_len != name_len:
                        warning = "Frame "+ str(frame_count)+" "+"Mismatch in number of names and units; use with caution."
                    else:
                        warning = ""
                    print(warning)
                curve_frames.append(curves_df)
    report = ("Number of frames = " + str(frame_count))  
    print(report)
        
    return curve_units, curve_names, curve_frames, wellname[0], warning, wellname[1], report

In [ ]:
filepath = r"C:\Users\aruss\OneDrive - Equinor\Desktop_2019\Volve_Well_logs_pr_WELL\15_9-F-4\04.COMPOSITE\WLC_PETROPHYSICAL_COMPOSITE_1.LIS"

In [ ]:
filepath = r"C:\Users\aruss\OneDrive - Equinor\Desktop_2019\Volve_Well_logs_pr_WELL\15_9-F-4\01.MUD_LOG\MUD_LOG_1.LIS"

In [ ]:
results = read_curves_and_curve_meta(filepath)

In [ ]:
results[0]

In [ ]:
results[1]

In [ ]:
results[2][0]

In [ ]:
results[3]

In [ ]:
##work in progress

def convert_to_las(filepath, output_folder_location, null=-999.25, curve_units=results[0], curve_names=results[1],
                  curves_df=results[3], wellname=results[3], warning=results[4]):
    
    filename = os.path.basename(filepath)
    filename = os.path.splitext(filename)[0]

